In [136]:
import numpy as np
import networkx as nx
import sys
sys.path.append("/Users/nando/Repos/DCOP/py_dcop/src/")

from core.dcop_instance import DCOPInstance
from utils.ccg_utils import transform_dcop_instance_to_ccg

data_path = '/Users/nando/Repos/DCOP/py_dcop/data/'
dcopIstance = DCOPInstance(data_path + 'binary.json')
G = transform_dcop_instance_to_ccg(dcopIstance)

print('weights:', nx.get_node_attributes(G, 'weight'))

weights: {'1': 30.0, '2': 22.0, '3': 12.0, '4': 12.0, '5': 8.0, '6': 9.0, '7': 13.0, '8': 10.0, '9': 3.0, '10': 4.0, '11': 3.0, '12': 2.0, '13': 4.0, '14': 4.0, '15': 5.0, '16': 8.0, '17': 9.0, '18': 4.0, '19': 5.0, '20': 1.0, '21': 2.0, '22': 3.0, '23': 4.0, '24': 2.0, '25': 3.0, '26': 2.0, '27': 3.0, '28': 2.0, '29': 3.0, '30': 1.0, '31': 2.0, '32': 2.0, '33': 3.0, '34': 8.0, '35': 9.0, '36': 8.0, '37': 9.0}


Running /Users/nando/Repos/DCOP/py_dcop/third_parties/wcsp/build/bin/wcsp -k -g /var/folders/q4/003xq7117nnfy9_cth24pwk80000gs/T/tmplrhj43jk


In [137]:
# Associates variables to CCG nodes
var_to_ccg_nodes = { vname : [] for vname in dcopIstance.variables}
for n, d in G.nodes(data=True):
    if 'variable' in d:
        var_to_ccg_nodes[d['variable']].append(n)

# Example to retrieve node from variable
for n in var_to_ccg_nodes['v0']:
    print(G.node[n])

{'weight': 30.0, 'type': 'decision', 'variable': 'v0', 'rank': 1}
{'weight': 22.0, 'type': 'decision', 'variable': 'v0', 'rank': 2}


In [172]:
def add_node_from(G_to, G_from, n):
    G_to.add_node(n)
    for attr in G_from.nodes[n]:
       G_to.nodes[n][attr] = G_from.nodes[n][attr]

In [184]:
## Partition the nodes among agents:
processed_nodes = []

G_agts = {vname: nx.Graph() for vname in dcopIstance.variables}

for v in dcopIstance.variables:
    for n in var_to_ccg_nodes[v]:
        add_node_from(G_agts[v], G, n)
        processed_nodes.append(n)
        
    for n in var_to_ccg_nodes[v]:
        ngbs_n = G.neighbors(n)
        for m in ngbs_n:
            if m not in processed_nodes:
                add_node_from(G_agts[v], G, m)
                processed_nodes.append(m)
                
    # todo: missing aux variables type 2 (neighbors of neighbors)?
assert(G.number_of_nodes() == len(processed_nodes))

In [185]:
## Partition edges among agents
processed_edges = []
for v in dcopIstance.variables:
    #print(nx.edges(G, G_agts[v].nodes))
    tmp = []
    for n in G_agts[v].nodes:
        for e in nx.edges(G, n):
            if (e[0], e[1]) in processed_edges: continue
            if (e[1], e[0]) in processed_edges: continue
            processed_edges.append((e[0], e[1]))
            processed_edges.append((e[1], e[0]))
            tmp.append(e)
    G_agts[v].add_edges_from(tmp)

assert (G.number_of_edges()*2 == len(processed_edges))

In [175]:
for v in dcopIstance.variables:
    for (n1, n2) in G_agts[v].edges:
        add_node_from(G_agts[v], G, n2)
        #G_agts[v].add_node(n2, attr_dict=G.node[n2])

In [178]:
nx.get_node_attributes(G, 'weight')

{'1': 30.0,
 '2': 22.0,
 '3': 12.0,
 '4': 12.0,
 '5': 8.0,
 '6': 9.0,
 '7': 13.0,
 '8': 10.0,
 '9': 3.0,
 '10': 4.0,
 '11': 3.0,
 '12': 2.0,
 '13': 4.0,
 '14': 4.0,
 '15': 5.0,
 '16': 8.0,
 '17': 9.0,
 '18': 4.0,
 '19': 5.0,
 '20': 1.0,
 '21': 2.0,
 '22': 3.0,
 '23': 4.0,
 '24': 2.0,
 '25': 3.0,
 '26': 2.0,
 '27': 3.0,
 '28': 2.0,
 '29': 3.0,
 '30': 1.0,
 '31': 2.0,
 '32': 2.0,
 '33': 3.0,
 '34': 8.0,
 '35': 9.0,
 '36': 8.0,
 '37': 9.0}

In [151]:
# G_agts2 = {vname: nx.Graph() for vname in dcopIstance.variables}
# 
# for var in dcopIstance.variables:
#     nodes = []
#     for v in var_to_ccg_nodes[var]:
#         if v not in nodes: nodes.append(v)
#         for u in G.neighbors(v):
#             if u not in nodes: nodes.append(u)
#     G_agts2[var] = nx.Graph(G.subgraph(nodes))